# Project 3: US Commodity Flow Survey

Group Members: Boer Fang, Chris Bunker, Patrick Steeves, Stella Lu 

# Part 1: Data Selection and Description

### Data Selection

For this project, our group decided to select data for US Commodities. The original dataset had over 4 million records in it which made it extremely slow to upload and analyze. We made the decision to reduce the number of records by just selecting the first 500,000 records. We believe this will still give a good sample for the year, especially because the data does not go in chronological order. By taking the first 500,000 we are taking a random sample.

The goal for this project is to see...

### Data Source

The original data was downloaded from the US Census: url
The reduced data we will use in this notebook can be found on github:

...

### Data Description

The medadata file for this data can be found here:

This data ... 

# <br><br>Part 2: Data Wrangling 

In [1]:
pwd

'/home/jovyan/work'

In [2]:
ls

CF.csv  Commodity_Flow_Survey.zip  Project_03_v4.ipynb


In [3]:
!unzip /home/jovyan/work/Commodity_Flow_Survey.zip

Archive:  /home/jovyan/work/Commodity_Flow_Survey.zip
  inflating: Commodity_Flow_Survey.csv  


In [1]:
!mv Commodity_Flow_Survey.csv CF.csv

In [3]:
!wc -l CF.csv

115 CF.csv


In [4]:
!csvcut -n CF.csv

  1: SHIPMT_ID
  2: ORIG_STATE
  3: ORIG_MA
  4: ORIG_CFS_AREA
  5: DEST_STATE
  6: DEST_MA
  7: DEST_CFS_AREA
  8: NAICS
  9: QUARTER
 10: SCTG
 11: MODE
 12: SHIPMT_VALUE
 13: SHIPMT_WGHT
 14: SHIPMT_DIST_GC
 15: SHIPMT_DIST_ROUTED
 16: TEMP_CNTL_YN
 17: EXPORT_YN
 18: EXPORT_CNTRY
 19: HAZMAT
 20: WGT_FACTOR


## <br><br>Commodity Flow Database

In [2]:
!echo 'redspot' | sudo -S service postgresql restart

[sudo] password for jovyan: Restarting PostgreSQL 9.5 database server: main.


In [3]:
!createdb -U dbuser proj3

In [4]:
%load_ext sql

/opt/conda/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/opt/conda/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [5]:
%sql postgresql://dbuser@localhost:5432/proj3

'Connected: dbuser@proj3'

In [6]:
%%sql
DROP TABLE IF EXISTS CF;
CREATE TABLE CF (
    SHIPMT_ID INTEGER,
    ORIG_STATE INTEGER,
    ORIG_MA INTEGER,
    ORIG_CFS_AREA VARCHAR (10),
    DEST_STATE INTEGER,
    DEST_MA INTEGER,
    DEST_CFS_AREA VARCHAR (10),
    NAICS INTEGER,
    QUARTER INTEGER,
    SCTG INTEGER,
    MODE INTEGER,
    SHIPMT_VALUE INTEGER,
    SHIPMT_WGHT INTEGER,
    SHIPMT_DIST_GC INTEGER,
    SHIPMT_DIST_ROUTED INTEGER,
    TEMP_CNTL_YN BOOLEAN,
    EXPORT_YN BOOLEAN,
    EXPORT_CNTRY CHAR (1),
    HAZMAT VARCHAR (1),
    WGT_FACTOR NUMERIC
);

Done.
Done.


[]

In [7]:
%%sql
COPY CF FROM '/home/jovyan/work/CF.csv'
CSV
HEADER
DELIMITER ',';

114 rows affected.


[]

In [8]:
%%sql
ALTER TABLE CF
DROP COLUMN SHIPMT_ID;

Done.


[]

In [9]:
%%sql
ALTER TABLE CF
ADD COLUMN SHIPMT_ID BIGSERIAL;

Done.


[]

In [10]:
%%sql
SELECT COUNT(*) FROM CF;

1 rows affected.


count
114


In [11]:
%%sql 
SELECT DISTINCT HAZMAT FROM CF;

3 rows affected.


hazmat
H
N
P


In [12]:
%%sql
SELECT orig_state, COUNT(*) AS count 
FROM CF
GROUP BY orig_state
ORDER BY count DESC
LIMIT 10;

10 rows affected.


orig_state,count
42,10
6,9
39,6
29,5
55,5
17,4
26,4
36,4
27,4
1,4


## <br><br>Create Star Schema

First we begin by creating our fact table and populating it with all of the data from the original CF table.

In [13]:
%%sql
DROP TABLE IF EXISTS cf_fact2;
CREATE TABLE cf_fact2 AS
    (SELECT * FROM CF);

Done.
114 rows affected.


[]

Next, we begin to create our dimensional tables and adding a primary key to each of them by altering the table after creating and adding a BIGSERIAL primary key identifier.

### <br>Control Dimension

In [14]:
%%sql
DROP TABLE IF EXISTS control_dimension;
CREATE TABLE control_dimension AS
    SELECT DISTINCT
    temp_cntl_yn,
    hazmat
FROM CF;

Done.
4 rows affected.


[]

In [15]:
%%sql
ALTER TABLE control_dimension ADD COLUMN control_key BIGSERIAL PRIMARY KEY;

Done.


[]

### <br>Update Fact2 Table

In [16]:
%%sql
DROP TABLE IF EXISTS cf_fact3;
CREATE TABLE cf_fact3 AS
(
SELECT f.shipmt_id, f.ORIG_STATE, f.ORIG_MA, f.ORIG_CFS_AREA, f.DEST_STATE, f.DEST_MA, f.DEST_CFS_AREA, f.NAICS, f.QUARTER,
    f.SCTG, f.MODE, f.SHIPMT_VALUE, F.SHIPMT_WGHT, SHIPMT_DIST_GC, SHIPMT_DIST_ROUTED, f.EXPORT_YN, f.EXPORT_CNTRY, f.WGT_FACTOR, d.control_key
FROM (SELECT * FROM control_dimension) AS d
INNER JOIN (SELECT * FROM cf_fact2) AS f
ON f.temp_cntl_yn = d.temp_cntl_yn
AND f.HAZMAT = d.hazmat
    )
;

Done.
114 rows affected.


[]

In [17]:
%%sql
SELECT * FROM cf_fact3
LIMIT 5;

5 rows affected.


shipmt_id,orig_state,orig_ma,orig_cfs_area,dest_state,dest_ma,dest_cfs_area,naics,quarter,sctg,mode,shipmt_value,shipmt_wght,shipmt_dist_gc,shipmt_dist_routed,export_yn,export_cntry,wgt_factor,control_key
1,25,148,25-148,25,148,25-148,333,2,35,14,2178,11,14,17,False,N,208.5,3
2,42,428,42-428,6,41740,06-41740,311,3,35,14,344,11,2344,2734,False,N,193.3,3
3,26,220,26-220,47,314,47-314,322,2,27,4,4197,5134,470,579,False,N,51.2,3
4,20,556,20-556,20,556,20-556,323,1,29,4,116,6,3,3,False,N,238.7,3
5,12,99999,12-99999,12,99999,12-99999,4235,3,33,5,388,527,124,201,False,N,398.1,3


### <br>Export Dimension

In [18]:
%%sql
DROP TABLE IF EXISTS export_dimension;
CREATE TABLE export_dimension AS
    SELECT DISTINCT
    export_yn,
    export_cntry
FROM CF;

Done.
3 rows affected.


[]

In [19]:
%%sql
ALTER TABLE export_dimension ADD COLUMN export_key BIGSERIAL PRIMARY KEY;

Done.


[]

### <br>Update Fact3 Table

In [20]:
%%sql
DROP TABLE IF EXISTS cf_fact4;
CREATE TABLE cf_fact4 AS
(
SELECT f.SHIPMT_ID, f.ORIG_STATE, f.ORIG_MA, f.ORIG_CFS_AREA, f.DEST_STATE, f.DEST_MA, f.DEST_CFS_AREA,
    f.NAICS, f.QUARTER, f.SCTG, f.MODE, f.SHIPMT_VALUE, f.SHIPMT_WGHT, f.SHIPMT_DIST_GC, f.SHIPMT_DIST_ROUTED, f.WGT_FACTOR,
    f.control_key, d.export_key
FROM (SELECT * FROM export_dimension) AS d
INNER JOIN (SELECT * FROM cf_fact3) AS f
ON d.export_yn = f.export_yn
AND f.export_cntry = d.export_cntry
    )
;

Done.
114 rows affected.


[]

In [21]:
%%sql
SELECT * FROM cf_fact4
LIMIT 5;

5 rows affected.


shipmt_id,orig_state,orig_ma,orig_cfs_area,dest_state,dest_ma,dest_cfs_area,naics,quarter,sctg,mode,shipmt_value,shipmt_wght,shipmt_dist_gc,shipmt_dist_routed,wgt_factor,control_key,export_key
1,25,148,25-148,25,148,25-148,333,2,35,14,2178,11,14,17,208.5,3,3
2,42,428,42-428,6,41740,06-41740,311,3,35,14,344,11,2344,2734,193.3,3,3
3,26,220,26-220,47,314,47-314,322,2,27,4,4197,5134,470,579,51.2,3,3
4,20,556,20-556,20,556,20-556,323,1,29,4,116,6,3,3,238.7,3,3
5,12,99999,12-99999,12,99999,12-99999,4235,3,33,5,388,527,124,201,398.1,3,3


### <br>Origin Dimension

In [22]:
%%sql
DROP TABLE IF EXISTS origin_dimension;
CREATE TABLE origin_dimension AS
    SELECT DISTINCT
    orig_state,
    orig_ma,
    orig_cfs_area
FROM CF;

Done.
69 rows affected.


[]

In [23]:
%%sql
ALTER TABLE origin_dimension ADD COLUMN origin_key BIGSERIAL PRIMARY KEY;

Done.


[]

### <br>Update Fact4 Table

In [24]:
%%sql
DROP TABLE IF EXISTS cf_fact5;
CREATE TABLE cf_fact5 AS
(
SELECT f.SHIPMT_ID, f.DEST_STATE, f.DEST_MA, f.DEST_CFS_AREA, f.NAICS, f.QUARTER, f.SCTG, f.MODE, f.SHIPMT_VALUE, 
    f.SHIPMT_WGHT, f.SHIPMT_DIST_GC, f.SHIPMT_DIST_ROUTED, f.WGT_FACTOR, f.control_key, f.export_key, d.origin_key
FROM (SELECT * FROM origin_dimension) AS d
INNER JOIN (SELECT * FROM cf_fact4) AS f
ON d.orig_state = f.orig_state
AND d.orig_ma = f.orig_ma
AND d.orig_cfs_area = f.orig_cfs_area
    )
;

Done.
114 rows affected.


[]

In [25]:
%%sql
SELECT * FROM cf_fact5
LIMIT 5;

5 rows affected.


shipmt_id,dest_state,dest_ma,dest_cfs_area,naics,quarter,sctg,mode,shipmt_value,shipmt_wght,shipmt_dist_gc,shipmt_dist_routed,wgt_factor,control_key,export_key,origin_key
1,25,148,25-148,333,2,35,14,2178,11,14,17,208.5,3,3,31
2,6,41740,06-41740,311,3,35,14,344,11,2344,2734,193.3,3,3,26
3,47,314,47-314,322,2,27,4,4197,5134,470,579,51.2,3,3,59
4,20,556,20-556,323,1,29,4,116,6,3,3,238.7,3,3,17
5,12,99999,12-99999,4235,3,33,5,388,527,124,201,398.1,3,3,15


### <br>Destination Dimension

In [26]:
%%sql
DROP TABLE IF EXISTS dest_dimension;
CREATE TABLE dest_dimension AS
    SELECT DISTINCT
    dest_state,
    dest_ma,
    dest_cfs_area
FROM CF;

Done.
67 rows affected.


[]

In [27]:
%%sql
ALTER TABLE dest_dimension ADD COLUMN dest_key BIGSERIAL PRIMARY KEY;

Done.


[]

### <br>Update Fact5 Table

In [28]:
%%sql
DROP TABLE IF EXISTS cf_fact6;
CREATE TABLE cf_fact6 AS
(
SELECT f.SHIPMT_ID, f.NAICS, f.QUARTER, f.SCTG, f.MODE, f.SHIPMT_VALUE, f.SHIPMT_WGHT, f.SHIPMT_DIST_GC, 
    f.SHIPMT_DIST_ROUTED, f.WGT_FACTOR, f.control_key, f.export_key, f.origin_key, d.dest_key
FROM (SELECT * FROM dest_dimension) AS d
INNER JOIN (SELECT * FROM cf_fact5) AS f
ON d.dest_state = f.dest_state
AND d.dest_ma = f.dest_ma
AND d.dest_cfs_area = f.dest_cfs_area
    )
;

Done.
114 rows affected.


[]

In [29]:
%%sql
SELECT * FROM cf_fact6
LIMIT 5;

5 rows affected.


shipmt_id,naics,quarter,sctg,mode,shipmt_value,shipmt_wght,shipmt_dist_gc,shipmt_dist_routed,wgt_factor,control_key,export_key,origin_key,dest_key
1,333,2,35,14,2178,11,14,17,208.5,3,3,31,30
2,311,3,35,14,344,11,2344,2734,193.3,3,3,26,42
3,322,2,27,4,4197,5134,470,579,51.2,3,3,59,4
4,323,1,29,4,116,6,3,3,238.7,3,3,17,18
5,4235,3,33,5,388,527,124,201,398.1,3,3,15,16


### <br> Shipment dimension

In [30]:
%%sql
DROP TABLE IF EXISTS shipment_dimension;
CREATE TABLE shipment_dimension AS
    SELECT DISTINCT
    naics,
    sctg,
    mode
FROM CF;

Done.
92 rows affected.


[]

In [31]:
%%sql
ALTER TABLE shipment_dimension ADD COLUMN shipment_key BIGSERIAL PRIMARY KEY;

Done.


[]

In [32]:
%%sql
DROP TABLE IF EXISTS cf_fact;
CREATE TABLE cf_fact AS
(
SELECT f.SHIPMT_ID, f.QUARTER, f.SHIPMT_VALUE, f.SHIPMT_WGHT, f.SHIPMT_DIST_GC, 
    f.SHIPMT_DIST_ROUTED, f.WGT_FACTOR, f.control_key, f.export_key, f.origin_key, f.dest_key, d.shipment_key
FROM (SELECT * FROM shipment_dimension) AS d
INNER JOIN (SELECT * FROM cf_fact6) AS f
ON d.naics = f.naics
AND d.sctg = f.sctg
AND d.mode = f.mode
    )
;

Done.
114 rows affected.


[]

In [33]:
%%sql
ALTER TABLE cf_fact
ADD FOREIGN KEY (control_key) REFERENCES control_dimension(control_key),
ADD FOREIGN KEY (export_key) REFERENCES export_dimension(export_key),
ADD FOREIGN KEY (origin_key) REFERENCES origin_dimension(origin_key),
ADD FOREIGN KEY (dest_key) REFERENCES dest_dimension(dest_key),
ADD FOREIGN KEY (shipment_key) REFERENCES shipment_dimension(shipment_key);

Done.


[]

In [34]:
%%sql
DROP TABLE cf_fact2;
DROP TABLE cf_fact3;
DROP TABLE cf_fact4;
DROP TABLE cf_fact5;
DROP TABLE cf_fact6;

Done.
Done.
Done.
Done.
Done.


[]

## <br> Classification tables for states, transported goods, industries, and modes of transport

In the data the states are reference by number (0-56). These numbers account for out of country (0), the 50 states, and DC. For some reason, theres are numbers missin (i.e. 3 and 7) so that is where the extra numbers come from. We are going to make a reference table for the origin and destination dimension tables so that we can look at the data using state names rather than reference numbers

In [35]:
%%sql
DROP TABLE IF EXISTS state_dimension;
CREATE TABLE state_dimension(
    state_id NUMERIC PRIMARY KEY,
    state_name VARCHAR (50)
    )
;

Done.
Done.


[]

In [36]:
%%sql
INSERT INTO state_dimension (state_id, state_name)
VALUES (0, 'Out 0f Country'), 
    (1, 'Alabama'),
    (2, 'Alaska'),
    (4, 'Arizona'),
    (5, 'Arkansas'),
    (6, 'California'),
    (8, 'Colorado'),
    (9, 'Connecticut'),
    (10, 'Delaware'),
    (11, 'DC'),
    (12, 'Florida'),
    (13, 'Georgia'),
    (15, 'Hawaii'),
    (16, 'Idaho'),
    (17, 'Illinois'),
    (18, 'Indiana'),
    (19, 'Iowa'),
    (20, 'Kansas'),
    (21, 'Kentucky'),
    (22, 'Louisiana'),
    (23, 'Maine'),
    (24, 'Maryland'),
    (25, 'Massachusetts'),
    (26, 'Michigan'),
    (27, 'Minnesota'),
    (28, 'Mississippi'),
    (29, 'Missouri'),
    (30, 'Montana'),
    (31, 'Nebraska'),
    (32, 'Nevada'),
    (33, 'New Hampshire'),
    (34, 'New Jersey'),
    (35, 'New Mexico'),
    (36, 'New York'),
    (37, 'North Carolina'),
    (38, 'North Dakota'),
    (39, 'Ohio'),
    (40, 'Oklahoma'),
    (41, 'Oregon'),
    (42, 'Pennsylvania'),
    (44, 'Rhode Island'),
    (45, 'South Carolina'),
    (46, 'South Dakota'),
    (47, 'Tennessee'),
    (48, 'Texas'),
    (49, 'Utah'),
    (50, 'Vermont'),
    (51, 'Virginia'),
    (53, 'Washington'),
    (54, 'West Virgina'),
    (55, 'Wisconsin'),
    (56, 'Wyoming');

52 rows affected.


[]

In [37]:
%%sql
SELECT *
FROM state_dimension
LIMIT 10;

10 rows affected.


state_id,state_name
0,Out 0f Country
1,Alabama
2,Alaska
4,Arizona
5,Arkansas
6,California
8,Colorado
9,Connecticut
10,Delaware
11,DC


In [38]:
%%sql
ALTER TABLE origin_dimension
ADD FOREIGN KEY (orig_state) REFERENCES state_dimension(state_id);

Done.


[]

In [39]:
%%sql
ALTER TABLE dest_dimension
ADD FOREIGN KEY (dest_state) REFERENCES state_dimension(state_id);

Done.


[]

<br>Now we can check whether our dimension was created properly

In [56]:
%%sql
SELECT cf_fact.shipmt_value, cf_fact.shipmt_wght, origin_dimension.orig_state, state_dimension.state_name
FROM origin_dimension, state_dimension, cf_fact
WHERE cf_fact.origin_key = origin_dimension.origin_key
AND origin_dimension.orig_state = state_dimension.state_id
LIMIT 20;

20 rows affected.


shipmt_value,shipmt_wght,orig_state,state_name
2178,11,25,Massachusetts
344,11,42,Pennsylvania
4197,5134,26,Michigan
116,6,20,Kansas
388,527,12,Florida
3716,1132,24,Maryland
43738,13501,19,Iowa
77,4,6,California
338,12826,19,Iowa
145,22,13,Georgia


<br> We use the same method to create a dimension that defines the codes for transported goods in the shipment dimension

In [41]:
%%sql
DROP TABLE IF EXISTS goods_classification;
CREATE TABLE goods_classification(
    class_id INTEGER PRIMARY KEY,
    class_descr VARCHAR (100)
    )
;

Done.
Done.


[]

In [42]:
%%sql
INSERT INTO goods_classification (class_id, class_descr)
VALUES
(01,'Animals and Fish (live)'),
(02,'Cereal Grains (includes seed'),
(03,'Agricultural Products (excludes Animal Feed, Cereal Grains, and Forage Products'),
(04,'Animal Feed, Eggs, Honey, and Other Products of Animal Origin'),
(05,'Meat, Poultry, Fish, Seafood, and Their Preparations'),
(06,'Milled Grain Products and Preparations, and Bakery Products'),
(07,'Other Prepared Foodstuffs, and Fats and Oils'),
(08,'Alcoholic Beverages and Denatured Alcohol'),
(09,'Tobacco Products'),
(10,'Monumental or Building Stone'),
(11,'Natural Sands'),
(12,'Gravel and Crushed Stone (excludes Dolomite and Slate)'),
(13,'Other Non-Metallic Minerals not elsewhere classified'),
(14,'Metallic Ores and Concentrates'),
(15,'Coal'),
(16,'Crude Petroleum'),
(17,'Gasoline, Aviation Turbine Fuel, and Ethanol (includes Kerosene, and Fuel Alcohols)'),
(18,'Fuel Oils (includes Diesel, Bunker C, and Biodiesel)'),
(19,'Other Coal and Petroleum Products, not elsewhere classified'),
(20,'Basic Chemicals'),
(21,'Pharmaceutical Products'),
(22,'Fertilizers'),
(23,'Other Chemical Products and Preparations'),
(24,'Plastics and Rubber'),
(25,'Logs and Other Wood in the Rough'),
(26,'Wood Products'),
(27,'Pulp, Newsprint, Paper, and Paperboard'),
(28,'Paper or Paperboard Articles'),
(29,'Printed Products'),
(30,'Textiles, Leather, and Articles of Textiles or Leather'),
(31,'Non-Metallic Mineral Products'),
(32,'Base Metal in Primary or Semi-Finished Forms and in Finished Basic Shapes'),
(33,'Articles of Base Metal'),
(34,'Machinery'),
(35,'Electronic and Other Electrical Equipment and Components, and Office Equipment'),
(36,'Motorized and Other Vehicles (includes parts)'),
(37,'Transportation Equipment, not elsewhere classified'),
(38,'Precision Instruments and Apparatus'),
(39,'Furniture, Mattresses and Mattress Supports, Lamps, Lighting Fittings, and Illuminated Signs'),
(40,'Miscellaneous Manufactured Products'),
(41,'Waste and Scrap (excludes of agriculture or food, see 041xx)'),
(43,'Mixed Freight'),
(99,'Missing Code'),
(00,'SCTG suppressed')
;

44 rows affected.


[]

In [43]:
%%sql
SELECT * FROM goods_classification
LIMIT 10;

10 rows affected.


class_id,class_descr
1,Animals and Fish (live)
2,Cereal Grains (includes seed
3,"Agricultural Products (excludes Animal Feed, Cereal Grains, and Forage Products"
4,"Animal Feed, Eggs, Honey, and Other Products of Animal Origin"
5,"Meat, Poultry, Fish, Seafood, and Their Preparations"
6,"Milled Grain Products and Preparations, and Bakery Products"
7,"Other Prepared Foodstuffs, and Fats and Oils"
8,Alcoholic Beverages and Denatured Alcohol
9,Tobacco Products
10,Monumental or Building Stone


In [44]:
%%sql
ALTER TABLE shipment_dimension
ADD FOREIGN KEY (sctg) REFERENCES goods_classification(class_id);

Done.


[]

<br> Now we define the codes for industry classifications which are present in the shipment dimension

In [45]:
%%sql
DROP TABLE IF EXISTS industry_classification;
CREATE TABLE industry_classification(
    industry_id INTEGER PRIMARY KEY,
    industry_descr VARCHAR (100)
    )
;

Done.
Done.


[]

In [49]:
%%sql
INSERT INTO industry_classification (industry_id, industry_descr)
VALUES
(212,'Mining (except oil and gas)'),
(311,'Food manufacturing'),
(312,'Beverage and tobacco product manufacturing'),
(313,'Textile mills'),
(314,'Textile product mills'),
(315,'Apparel manufacturing'),
(316,'Leather and allied product manufacturing'),
(321,'Wood product manufacturing'),
(322,'Paper manufacturing'),
(323,'Printing and related support activities'),
(324,'Petroleum and coal products manufacturing'),
(325,'Chemical manufacturing'),
(326,'Plastics and rubber products manufacturing'),
(327,'Nonmetallic mineral product manufacturing'),
(331,'Primary metal manufacturing'),
(332,'Fabricated metal product manufacturing'),
(333,'Machinery manufacturing'),
(334,'Computer and electronic product manufacturing'),
(335,'Electrical equipment, appliance, and component manufacturing'),
(336,'Transportation equipment manufacturing'),
(337,'Furniture and related product manufacturing'),
(339,'Miscellaneous manufacturing'),
(4231,'Motor vehicle and parts merchant wholesalers'),
(4232,'Furniture and home furnishing merchant wholesalers'),
(4233,'Lumber and other construction materials merchant wholesalers'),
(4234,'Commercial equip. merchant wholesalers'),
(4235,'Metal and mineral (except petroleum) merchant wholesalers'),
(4236,'Electrical and electronic goods merchant wholesalers'),
(4237,'Hardware and plumbing merchant wholesalers'),
(4238,'Machinery, equipment, and supplies merchant wholesalers'),
(4239,'Miscellaneous durable goods merchant wholesalers'),
(4241,'Paper and paper product merchant wholesalers'),
(4242,'Drugs and druggists'' sundries merchant wholesalers'),
(4243,'Apparel, piece goods, and notions merchant wholesalers'),
(4244,'Grocery and related product merchant wholesalers'),
(4245,'Farm product raw material merchant wholesalers'),
(4246,'Chemical and allied products merchant wholesalers'),
(4247,'Petroleum and petroleum products merchant wholesalers'),
(4248,'Beer, wine, and distilled alcoholic beverage merchant wholesalers'),
(4249,'Miscellaneous nondurable goods merchant wholesalers'),
(4541,'Electronic shopping and mail-order houses'),
(45431,'Direct selling establishments'),
(4931,'Warehousing and storage (includes 484)'),
(5111,'Newspaper, periodical, book, and directory publishers'),
(551114,'Corporate, subsidiary, and regional managing offices')
;

45 rows affected.


[]

<br> Finally, we create a table that defines the codes for modes of transport

In [50]:
%%sql
DROP TABLE IF EXISTS transport_mode;
CREATE TABLE transport_mode(
    mode_id INTEGER PRIMARY KEY,
    mode_descr VARCHAR (50)
    )
;

Done.
Done.


[]

In [52]:
%%sql
INSERT INTO transport_mode 
VALUES
(02,'Single mode'),
(03,'Truck'),
(04,'For-hire truck'),
(05,'Private truck'),
(06,'Rail'),
(07,'Water'),
(08,'Inland Water'),
(09,'Great Lakes'),
(10,'Deep Sea'),
(101,'Multiple Waterways'),
(11,'Air (incl truck & air)'),
(12,'Pipeline'),
(13,'Multiple mode'),
(14,'Parcel, USPS, or courier'),
(20,'Non-parcel multimode'),
(15,'Truck and rail'),
(16,'Truck and water'),
(17,'Rail and water'),
(18,'Other multiple mode'),
(19,'Other mode'),
(00,'Mode suppressed')
;


21 rows affected.


[]

In [54]:
%%sql
ALTER TABLE shipment_dimension
ADD FOREIGN KEY (mode) REFERENCES transport_mode(mode_id);

Done.


[]